In [23]:
pip install --upgrade "yfinance>=0.2.0,<0.3.0"

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.1.96
    Uninstalling yfinance-0.1.96:
      Successfully uninstalled yfinance-0.1.96
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
from time import sleep
import yfinance as yf
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
chromedriver_autoinstaller.install()

'C:\\Users\\okand\\anaconda3\\Lib\\site-packages\\chromedriver_autoinstaller\\139\\chromedriver.exe'

In [9]:
df=pd.read_excel("piotroski_f_scores.xlsx")
df.head(10)

,Kod,f_score
0,LOGO.IS,9
1,GSRAY.IS,8
2,KARSN.IS,7
3,SARKY.IS,7
4,ECILC.IS,7
5,BFREN.IS,7
6,TKFEN.IS,7
7,ADEL.IS,6
8,GUBRF.IS,6
9,DOAS.IS,6


In [11]:
tickers=df["Kod"][(df["f_score"]>=6)].tolist()

In [15]:
path = "C:\\Users\\okand\\anaconda3\\Lib\\site-packages\\chromedriver_autoinstaller\\139\\chromedriver.exe"

In [17]:
dropping=["Highlights","Year","Profitability","Effectiveness","Statement","Sheet","Information","Statistics","Splits",
         "Guide","Quarter","Months","Year","Ending","",np.nan]

In [19]:
stats = ["EBITDA",
         "DepreciationAmortizationDepletion",
         "Market Cap",
         "NetIncome",
         "OperatingCashFlow",
         "CapitalExpenditure",
         "CurrentAssets",
         "CurrentLiabilities",
         "NetPPEPurchaseAndSale",
         "StockholdersEquity",
         "LongTermDebtAndCapitalLeaseObligation",
         "Forward Annual Dividend Yield 4"] # change as required

indx = ["EBITDA","D&A","MarketCap","NetIncome","CashFlowOps","Capex","CurrAsset",
        "CurrLiab","PPE","BookValue","TotDebt","DivYield"]

In [21]:
def get_statistic(ticker):
    
    temp_dir = {}
    url="https://finance.yahoo.com/quote/{}/key-statistics/".format(ticker)
    service = webdriver.chrome.service.Service(path)
    service.start()
    options = Options()
    options.add_argument("--headless") #use this to not render the actual browser    
    driver = webdriver.Chrome(service=service, options = options)
    driver.get(url)
    
    WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,
                                                                        '//*[@id="nimbus-app"]/section/section/section/article/section[2]/div/table')))
    table = driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/section[2]/div/table')
    for cell in table:          
        vals = cell.text.split("\n")
        for val in vals:
            values = val.split(" ")
            if values[0] == "Current":
                pass
            else:
                temp_dir[" ".join(values[:-6])] = values[-6].replace('M', 'E+03').replace('B', 'E+06').replace('T', 'E+09').replace('%', 'E-02')

    WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,
                                                                        '//*[@id="nimbus-app"]/section/section/section/article/article/div/section[1]')))
    table2 = driver.find_elements(By.XPATH,'//*[@id="nimbus-app"]/section/section/section/article/article/div/section[1]')
    for cell in table2:
        values = cell.text.split("\n")
        for val in values:
            values2=val.split(" ")
            if values2[-1] in dropping or values2[0] in ["EBITDA"]:
                continue
            else:
                temp_dir[" ".join(values2[:-1])] = values2[-1].replace('M', 'E+03').replace('B', 'E+06').replace('T', 'E+09').replace('%', 'E-02')

    WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,
                                                                        '//*[@id="nimbus-app"]/section/section/section/article/article/div/section[2]')))
    table3= driver.find_elements(By.XPATH,'//*[@id="nimbus-app"]/section/section/section/article/article/div/section[2]')
    for cell in table3:
        values = cell.text.split("\n")
        for val in values:
            values2=val.split(" ")
            if values2[-1] in dropping:
                continue
            else:
                temp_dir[" ".join(values2[:-1])] = values2[-1].replace('M', 'E+03').replace('B', 'E+06').replace('T', 'E+09').replace('%', 'E-02')
    
    df_samp = pd.DataFrame(temp_dir.values(),index=temp_dir.keys(),columns=[ticker])
    #df_samp.iloc[:,0] = df_samp.iloc[:,0].replace({'M': 'E+03','B': 'E+06','T': 'E+09','%': 'E-02'}, regex=True)
    #df_samp.iloc[:,0] = pd.to_numeric(df_samp.iloc[:,0],errors="coerce")
    sleep(2)
    driver.close()
    return df_samp

In [23]:
def get_DAD(ticker):
    hisseler = [ticker.split(".IS")[0]]
    for h in hisseler:
        hisse=h
        tarihler=[]
        yıllar=[]
        donemler=[]
        grup=[]
    
        url1="https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/sirket-karti.aspx?hisse="+hisse
        r1=requests.get(url1)
        soup=BeautifulSoup(r1.text,"html.parser")
        secim=soup.find("select",id="ddlMaliTabloFirst")
        secim2=soup.find("select",id="ddlMaliTabloGroup")
        
        try:
            cocuklar=secim.findChildren("option")
            grup=secim2.find("option")["value"]
            
            for i in cocuklar:
                tarihler.append(i.string.rsplit("/"))
            for j in tarihler:
                yıllar.append(j[0])
                donemler.append(j[1])
    
            if len(tarihler)>=4:
                parametreler=(
                    ("companyCode",hisse),
                    ("exchange","TRY"), #Eğer dolar bazlı istiyorsanız "TRY" yerine "USD" yazınız
                    ("financialGroup",grup),
                    ("year1",yıllar[0]),
                    ("period1",donemler[0]),
                    ("year2",yıllar[1]),
                    ("period2",donemler[1]),
                    ("year3",yıllar[2]),
                    ("period3",donemler[2]),
                    ("year4",yıllar[3]),
                    ("period4",donemler[3]))
                url2="https://www.isyatirim.com.tr/_layouts/15/IsYatirim.Website/Common/Data.aspx/MaliTablo"
                r2=requests.get(url2,params=parametreler).json()["value"]
                veri=pd.DataFrame.from_dict(r2)
                veri.drop(columns=["itemCode","itemDescEng"],inplace=True)           
            else:
                continue
        except AttributeError:
            continue
        
        del tarihler[0:4]
        tumveri=[veri]
            
        for _ in range(0,int(len(tarihler)+1)):
            if len(tarihler)==len(yıllar):
                del tarihler[0:4]
            else:
                yıllar=[]
                donemler=[]
                for j in tarihler:
                    yıllar.append(j[0])
                    donemler.append(j[1])
                
                if len(tarihler)>=4:
                    parametreler2=(
                    ("companyCode",hisse),
                    ("exchange","TRY"), #Eğer dolar bazlı istiyorsanız "TRY" yerine "USD" yazınız
                    ("financialGroup",grup),
                    ("year1",yıllar[0]),
                    ("period1",donemler[0]),
                    ("year2",yıllar[1]),
                    ("period2",donemler[1]),
                    ("year3",yıllar[2]),
                    ("period3",donemler[2]),
                    ("year4",yıllar[3]),
                    ("period4",donemler[3]))
                    r3=requests.get(url2,params=parametreler2).json()["value"]
                    veri2=pd.DataFrame.from_dict(r3)
                    try:
                        veri2.drop(columns=["itemCode","itemDescTr","itemDescEng"],inplace=True)
                        tumveri.append(veri2)
                    except KeyError:
                        continue
        veri3=pd.concat(tumveri,axis=1)
        baslık=["Bilanço"]
        for i in cocuklar:
            baslık.append(i.string)
    
        baslıkfark=len(baslık)-len(veri3.columns)
    
        if baslıkfark!=0:
            del baslık[-baslıkfark:]
    
        veri3=veri3.set_axis(baslık,axis=1)
        veri3[baslık[1:]]=veri3[baslık[1:]].astype(float)
        veri3=veri3.fillna(0)
        veri3.set_index("Bilanço",inplace=True)
        amort=veri3.loc["Amortisman Giderleri",veri3.columns[0]]
        return amort


In [ ]:
financial_dir = {}

for ticker in tickers:
    try:
        t = yf.Ticker(ticker)

        df1 = t.income_stmt.iloc[:, 0].to_frame(name=ticker)

        df2 = t.balance_sheet.iloc[:, 0].to_frame(name=ticker)

        df3 = t.cashflow.iloc[:, 0].to_frame(name=ticker)

        df4 = get_statistic(ticker)
        # df4 = df4.to_frame(name=ticker)  # Eğer df4 Series ise

        # Hepsini birleştir
        df = pd.concat([df1, df2, df3, df4])

        # Eğer amortisman yoksa ekle
        if "DepreciationAmortizationDepletion" not in df.index:
            df.loc["DepreciationAmortizationDepletion", ticker] = get_DAD(ticker)

        financial_dir[ticker] = df
        print("data extracted for", ticker)

    except Exception as e:
        print(ticker, ":", e)

LOGO.IS : Message: 



In [21]:
financial_dir["ASELS.IS"].loc["DepreciationAmortizationDepletion",:]

KeyError: 'ASELS.IS'

In [ ]:
financial_dir2 = {}

for ticker in financial_dir.keys():
    vals = []
    idx_list = financial_dir[ticker].index.tolist()
    first_col = financial_dir[ticker].columns[0]  # ilk kolonun ismini al
    for i in idx_list:
        if financial_dir[ticker].loc[i, ticker] is np.nan:
            vals.append(financial_dir[ticker].loc[i, first_col])
        else:
            vals.append(financial_dir[ticker].loc[i, ticker])
    df = pd.DataFrame(vals, index=idx_list, columns=[ticker])
    financial_dir2[ticker] = df

In [ ]:
def info_filter(df,stats,indx):
    """function to filter relevant financial information
       df = dataframe to be filtered
       stats = headings to filter
       indx = rename long headings
       lookback = number of years of data to be retained"""
    for stat in stats:
        if stat not in df.index:
            print("unable to find {} in {}".format(stat,df.columns[0]))
            return
    df_new = df.loc[stats]
    df_new = df_new[~df_new.index.duplicated(keep='first')]
    df_new.iloc[:,0]=pd.to_numeric(df_new.iloc[:,0],errors="coerce")
    df_new.rename(dict(zip(stats,indx)),inplace=True)
    return df_new

In [ ]:
transformed_df = {}
for ticker in financial_dir:
    transformed_df[ticker] = info_filter(financial_dir2[ticker],stats,indx)
    if transformed_df[ticker] is None:
        del transformed_df[ticker]
        continue
    transformed_df[ticker].loc["EBIT",:] = transformed_df[ticker].loc["EBITDA",:] - transformed_df[ticker].loc["D&A",:]
    transformed_df[ticker].loc["TEV",:] =  transformed_df[ticker].loc["MarketCap",:] + \
                                           transformed_df[ticker].loc["TotDebt",:] - \
                                           (transformed_df[ticker].loc["CurrAsset",:]-transformed_df[ticker].loc["CurrLiab",:])
    transformed_df[ticker].loc["EarningYield",:] =  transformed_df[ticker].loc["EBIT",:]/transformed_df[ticker].loc["TEV",:]
    transformed_df[ticker].loc["FCFYield",:] = (transformed_df[ticker].loc["CashFlowOps",:]-transformed_df[ticker].loc["Capex",:])/transformed_df[ticker].loc["MarketCap",:]
    transformed_df[ticker].loc["ROC",:]  = (transformed_df[ticker].loc["EBITDA",:] - transformed_df[ticker].loc["D&A",:])/(transformed_df[ticker].loc["PPE",:]+transformed_df[ticker].loc["CurrAsset",:]-transformed_df[ticker].loc["CurrLiab",:])
    transformed_df[ticker].loc["BookToMkt",:] = transformed_df[ticker].loc["BookValue",:]/transformed_df[ticker].loc["MarketCap",:]

In [ ]:
final_stats_val_df = pd.DataFrame(columns=transformed_df.keys())
for key in final_stats_val_df:
    final_stats_val_df[key] = transformed_df[key].values.flatten()
final_stats_val_df.set_index(transformed_df[key].index,inplace=True)

final_stats_val_df

In [ ]:
# finding value stocks based on Magic Formula

final_stats_val_df.loc["ComRank",:] = final_stats_val_df.loc["EarningYield",:].rank(ascending=False,na_option="bottom") + final_stats_val_df.loc["ROC",:].rank(ascending=False,na_option="bottom")
final_stats_val_df.loc["MagicFormulaRank"] = final_stats_val_df.loc["ComRank",:].rank(method="first")
value_stocks = final_stats_val_df.loc["MagicFormulaRank",:].sort_values()
print("----------------------")
print("Value stocks based on Greenblatt's Magic Formula")
print(value_stocks)

In [ ]:
# finding highest dividend yield stocks
high_dividend_stocks = final_stats_val_df.loc["DivYield",:].sort_values(ascending=False)
print("------------------------------------------------")
print("Highest dividend paying stocks")
print(high_dividend_stocks)

In [ ]:
# # Magic Formula & Dividend yield combined
final_stats_val_df.loc["CombinedRank",:] = final_stats_val_df.loc["EarningYield",:].rank(ascending=False, method="first") \
                                            + final_stats_val_df.loc["ROC",:].rank(ascending=False, method="first") \
                                            + final_stats_val_df.loc["DivYield"].rank(ascending=False,method="first")

value_high_div_stocks = final_stats_val_df.T.sort_values("CombinedRank").loc[:,["EarningYield","ROC","DivYield","CombinedRank"]]
print("------------------------------------------------")
print("Magic Formula and Dividend Yield combined")
print(value_high_div_stocks)